<h1> <center> ENSF 519.01 Applied Data Scince </center></h1>
<h2> <center> Assignment 5: Bug Localization (25 marks)</center></h2>
<h2> <center> Due: April 13, 4pm . To be submitted on D2L Dropbox </center></h2>


In this assignment, we are going to provide an automated system that given a bug report and the history of bug reports, suggests what files are likely to be buggy and need to be fixed. This problem in the software engineering domain is called Fault Localization and there are several solutions for it. Our approach will be based on TF.IDF and topic vector representations of the bug reports.


<h2> Step1: Preprocessing: Tokenizing - removing stopwords - stemming - vectorizing (7 marks)</h2>
<br><br>

Our dataset consists of a potion of Eclipse project's bug reports collected from. 

Link to Data: https://zenodo.org/record/268425#.WrVqNdPOVTZ

For each bug report the dataset have the following items: 

<ul>
<li> id: The ID which is assigned to each bug report starting from 1.
<li> bug_id: The bug report ID in the Eclipse company. 
<li> summary: A small description of bug in one line
<li> report_time: The date and time of the arrival of the bug report
<li> report_timestamp: A timeStamp assigned to each bug report when they arrive
<li> status: shows the status of bug at the time being. 
<li> commit: commit number.
<li> commit_timestamp: the commit time.
<li> files: files that have been modified to fix the bug
<li> description: The actual text of bug report. 
</ul>

We have created a shortened and cleaned version of the reports containing 100 bug reports for you with the following features:

<ul>
<li> Summary
<li> Description 
<li> Files
</ul>

All these bugs are "Closed" bug repots (they are already fixed)

Steps: 
<ol>
<li> Read the data from Eclipse_Platform_UI_100.csv 
<li> For each bug report in the dataset, take the Description + Summary as the document
<li> For each Document in the corpus, also keep a list of Files modified to fix the bug in a list, per bug. 
<li> Tokenize the Documents 
<li> Remove the stopwords from the Documents (Use SKLearn's stopword list)
<li> Stem the remaining using PorterStemmer (from nltk). 
<li> Finally, vectorize the corpus documents using CountVectorizer.
</ol>



In [37]:
#
# Imports
#
# pandas v0.22.0
import pandas as pd

# numpy v1.13.3
import numpy as np

# sklearn v0.19.1
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

# matplotlib v2.1.2
import matplotlib.pyplot as plt
%matplotlib inline

# nltk v3.2.5
from nltk.stem import PorterStemmer


#
# Settings
#
pd.set_option('display.max_colwidth',2500)


#
# Read the data from Eclipse_Platform_UI_100.csv
#
eclipse_path = "Eclipse_Platform_UI_100.csv"
try:
    eclipse_df = pd.read_csv(eclipse_path)
except FileNotFoundError as e:
    print(e)


#
# For each bug report in the dataset, take the Description + Summary as the document
#
data_df = pd.DataFrame()
data_df["document"] = eclipse_df["summary"] + eclipse_df["description"]


#
# For each Document in the corpus, also keep a list of Files modified to fix the bug in a list, per bug.
#
data_df["files"] = eclipse_df["files"].apply(lambda x: x.split())


#
# Tokenize the Documents
#
tokenizer = CountVectorizer().build_tokenizer()
data_df["tokens"] = data_df["document"].apply(lambda x: tokenizer(x))


#
# Remove the stopwords from the Documents (Use SKLearn's stopword list)
#
data_df["tokens_stopped"] = data_df["tokens"].apply(
    lambda x: [t.lower() for t in x if t.lower() not in ENGLISH_STOP_WORDS])


#
# Stem the remaining using PorterStemmer (from nltk).
#
stemmer = PorterStemmer()
data_df["tokens_stemmed"] = data_df["tokens_stopped"].apply(
    lambda x: [stemmer.stem(t) for t in x])
print("\n\nSTEMMED\n\n")
print(data_df.iloc[1]["tokens_stemmed"])


#
# Finally, vectorize the corpus documents using CountVectorizer.
#
all_tokens = [" ".join(token_list) for token_list in data_df["tokens_stemmed"]]
vect = CountVectorizer()
X = vect.fit_transform(all_tokens)



# print(type(p[0]))

# data_df["tokens_vectorized"] = p

# # data_df["tokens_vectorized"] = data_df["tokens_stemmed"].apply(
# #     lambda x: vect.transform(" ".join(x)))


# print("\n\nVECTORIZED\n\n")
# print(data_df.iloc[1]["tokens_vectorized"].toarray())



STEMMED


['bug', '382839', 'compat', 'line', 'line', 'longer', 'work', 'content', 'assist', 'pop', 'up4', 'rc4', 'line', 'line', 'longer', 'work', 'content', 'assist', 'pop', 'work', 'fine', 'rc4', 'step', 'start', 'new', 'workspac', 'assign', 'new', 'key', 'bind', 'line', 'past', 'class', 'packag', 'explor', 'invok', 'content', 'assist', 'don', 'focu', 'pop', 'use', 'key', 'bind', 'step', 'move', 'select', 'pop', 'expect', 'move', 'insid', 'editor', 'code', 'instal', 'handler', 'org', 'eclips', 'jface', 'text', 'contentassist', 'contentassist', 'instal']
<class 'scipy.sparse.csr.csr_matrix'>


VECTORIZED


[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


<h2> Step 2: LDA Model (10 marks)</h2>
<br><br>
In this step, we will apply an LDA model on the corpus documents and represent them using topic membership vectors. We then use this vectors to measure similarity between a document from the test set (query) and the train set documents. Then we score the train set documents according to their similarities to the query. Finally, we identify the files relevant to the query based on the modified files from the similar bug reports.

Since the documents have timestapms (bugs come and get fixed one after the other), to evaluate our approach, the best way is to use a time series cross-validation technique from SKLearn, but for the sake of simplicity we only take one train_test_split (20 vs. 80).

Steps to follow: 
<ol>
<li> Divide the corpus into a test and train set as follows: take the first 20 documents as test set and the rest as the training set.
<li> Using SKLearn, build an LDA model using the Bag of Words vectors of the documents, created in the previous question (with number of topics = 100). - Note that we give the entire train and test to LDA to build a model. This is not information leaking since we are doing unsupervised decomposition here and the LDA does not uses any information that is from future.
<li> For each document in both train and the test sets, find the topic membership vector (Vector of Probability). Vector of Probability is a vector that indicates the probability of the document to be a member of each topic. Now to compare two documents, we need to compare their Vectors of Probability. 
<li> For each test set document (query document):
  <ol>
  <li>apply "Hellinger Distance" (which is a suitable distance function for Vectors with values between 0 and 1.0) to calculate it's distance to every train set document. You can import hellinger from gensim.matutils </li>
  <li>assign the inverse of Hellinger Distance per training set document as its "Similarity Score"</li>
  <li>For each file, from the list of files modified in any bug fix in the training set (provided as the "files" attribute per bug report), calculate a Score, by summing Similarity Scores of all the training set documents, in which the file is seen. </li>
  <li>rank files based on the calculated Scores. </li>
  <li>pick a cut-off Threshold and form a predicted list of files (with Threshold = 5, 10, 15, 20, 25) </li>
  <li>Calculate Precision and Recall by comparing the predicted list with the actual list of modified files per each bug report in the test set. </li>
</ol>
<li> Report median Precision and Recall over all 20 test bug report
<li> Plot the Precision and Recall for different values of threshold
</ol>


In [2]:
from sklearn.decomposition import LatentDirichletAllocation

# Divide the corpus into a test and train set as follows: take the first 20 documents
# as test set and the rest as the training set.


# How to do this? Maybe we just grab the first 20 from the df?
# Looks like we need "bag of words" vectors for the documents



# Using SKLearn, build an LDA model using the Bag of Words vectors of the documents,
# created in the previous question (with number of topics = 100). - Note that we give
# the entire train and test to LDA to build a model. This is not information leaking
# since we are doing unsupervised decomposition here and the LDA does not uses any
# information that is from future.
lda = LatentDirichletAllocation(n_topics=100)
document_topics = lda.fit_transform(X)



# For each document in both train and the test sets, find the topic membership vector
# (Vector of Probability). Vector of Probability is a vector that indicates the
# probability of the document to be a member of each topic. Now to compare two documents,
# we need to compare their Vectors of Probability.

# For each test set document (query document):

# apply "Hellinger Distance" (which is a suitable distance function for Vectors with values
# between 0 and 1.0) to calculate it's distance to every train set document. You can import
# hellinger from gensim.matutils

# assign the inverse of Hellinger Distance per training set document as its "Similarity Score"

# For each file, from the list of files modified in any bug fix in the training set (provided
# as the "files" attribute per bug report), calculate a Score, by summing Similarity Scores
# of all the training set documents, in which the file is seen.

# rank files based on the calculated Scores.

# pick a cut-off Threshold and form a predicted list of files (with Threshold = 5, 10, 15, 20, 25)

# Calculate Precision and Recall by comparing the predicted list with the actual list of
# modified files per each bug report in the test set.

# Report median Precision and Recall over all 20 test bug report

# Plot the Precision and Recall for different values of threshold

<b> Question:</b> Which evaluation metric (Recall or Precision) is more important in this context? What is the easiest way to improve that measure? What is the practical problem with that way of improvement?
    

<b>Your Answer:</b> ...................

<h2> Step 3: TF.IDF rescaling (8 marks) </h2>
<br>In this part, we replicate the previous part but this time using a new distance function that we directly apply on TF.IDF-based representation of vectorized documents and NOT the topic memberships.  

Steps to follow: 
<ol>
<li> Using TF.IDF covert the vectorized documents from Step 1 to a matrix of doc-terms, where the cell value at (i,j) is the frequency of term j in doc i, multiplied by the idf of term j across the corpus.
<li> Now to compare two documents (two rows in the matrix) you can use Cosine Similarity measure. Since SKLearn already normalizes the TF.IDF values so that each document has a length of UNIT, you can simply take the Numpy's "Inner Product" of two rows (arrays) as their Cosine Similarity score. See https://en.wikipedia.org/wiki/Cosine_similarity for more details on the Cosine Similarity function. 
<li> After you have the Similarity Scores the rest is exactly like previous Step, at 4.C. So you basically will output the similar Precision-Recall plot.
</ol>




In [3]:
# Your code here

<b> Question: </b> Given that both LDA and TF.IDF provide low quality results, explian what can be the main reason? 
Note that you do not need to implment anything extra here. <br>
This is an open question with no definite answer.

<b>Your Answer:</b> ...................